In [ ]:
# Google Colabでの設定
google_colab = True

if google_colab:
    from google.colab import drive
    from google.colab import userdata

    drive.mount("/content/drive")

    # ディレクトリ移動
    %cd /content/drive/MyDrive/Python/kaggle_map/src/exp030_qwen2.5-32b-lora-softlabel-fold4

    !pip install -q -U trl==0.23.0
    !pip install -q accelerate==1.10.1 bitsandbytes==0.47.0 mpi4py==4.1.0 deepspeed==0.17.6 transformers==4.56.2

In [ ]:
import os
import gc
import time
import json
import random
import pickle
from tqdm import tqdm

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

import torch
from torch import nn
import torch.nn.functional as F
import wandb
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    PreTrainedTokenizer,
    Trainer
)
from peft import LoraConfig
from trl import SFTConfig, SFTTrainer
from trl.trainer.sft_trainer import DataCollatorForLanguageModeling

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# DeepSpeed requires a distributed environment even when only one process is used.
# This emulates a launcher in the notebook
os.environ["MASTER_ADDR"] = "localhost"
os.environ["MASTER_PORT"] = "9995"  # modify if RuntimeError: Address already in use
os.environ["RANK"] = "0"
os.environ["LOCAL_RANK"] = "0"
os.environ["WORLD_SIZE"] = "1"

In [ ]:
class CFG:
    """実験設定管理クラス"""

    # ============== 実験情報 =============
    comp_name = "kaggle_map"
    exp_name = "exp030_qwen2.5-32b-lora-softlabel-fold4"
    model_name = "Qwen/Qwen2.5-32B-Instruct"
    fold = 4

    # ============== ファイルパス設定 =============
    comp_dir_path = "../../kaggle/input/"
    comp_dataset_path = f"{comp_dir_path}/map-charting-student-math-misunderstandings/"
    soft_label_path = f"{comp_dir_path}/qwen2.5-32b/validation_logprobs_v2_4.pkl"
    output_dir_path = "output/"
    log_dir_path = "logs/"

    # ============== モデル設定 =============
    max_len = 256
    num_train_epochs = 2
    per_device_train_batch_size = 8
    gradient_accumulation_steps = 2
    per_device_eval_batch_size = 8
    optim_type = "adamw_torch"
    learning_rate = 1e-4
    lr_scheduler_type = "cosine"
    warmup_ratio = 0.03
    weight_decay = 0.01

    lora_r = 32
    lora_alpha = 64
    lora_dropout = 0.01
    lora_bias = "none"
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]
    task_type = "CAUSAL_LM"

    cols = ["prompt", "completion", "soft_labels"]

    # ============== ソフトラベル設定 =============
    temperature = 2.0
    alpha = 0.9

    # ============== その他設定 =============
    seed = 26
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # ============== プロンプト設定 =============
    prompt_format = """Question: {QuestionText}
Answer: {MC_Answer}
Correct: {Correct}
Student Explanation: {StudentExplanation}
Label: """

In [ ]:
# 乱数固定
def set_seed(seed=None, cudnn_deterministic=True):
    if seed is None:
        seed = 42

    os.environ["PYTHONHASHSEED"] = str(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = cudnn_deterministic
    torch.backends.cudnn.benchmark = False

def make_dirs(cfg):
    for dir in [cfg.output_dir_path, cfg.log_dir_path]:
        os.makedirs(dir, exist_ok=True)

def cfg_init(cfg):
    set_seed(cfg.seed)
    make_dirs(cfg)

In [ ]:
cfg_init(CFG)

## データの読み込みと前処理

In [ ]:
def add_folds_by_qid_cat_misc(df, n_splits=5, random_state=42, fallback="pair"):
    s_qid = df["QuestionId"].astype(str).fillna("NA")
    s_cat = df["Category"].astype(str).fillna("NA")
    s_misc = df["Misconception"].astype(str).fillna("NA")

    y_triple = s_qid + "|" + s_cat + "|" + s_misc
    y_pair = s_cat + "|" + s_misc

    cnt = y_triple.value_counts()
    if (cnt < n_splits).any():
        if fallback == "pair":
            rare = y_triple.map(cnt) < n_splits
            y = np.where(rare, y_pair, y_triple)
        elif fallback == "category":
            rare = y_triple.map(cnt) < n_splits
            y = np.where(rare, s_cat, y_triple)
        elif fallback == "none":
            y = y_triple
        else:
            raise ValueError("fallback は 'pair' / 'category' / 'none' のいずれかにしてください。")
    else:
        y = y_triple

    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    folds = np.full(len(df), -1, dtype=int)
    for fold, (_, val_idx) in enumerate(skf.split(np.zeros(len(df)), y)):
        folds[val_idx] = fold

    out = df.copy()
    out["fold"] = folds
    return out

In [ ]:
def wrong_corrections(df: pd.DataFrame) -> pd.DataFrame:
    """既知の誤りを修正する"""
    false_to_true_ids = [12878, 12901, 13876, 14089, 14159, 14185]
    df["MC_Answer"] = np.where(
        df["row_id"].isin(false_to_true_ids),
        df["MC_Answer"].str.replace(r"\( 6 \)", r"\( 9 \)"),
        df["MC_Answer"]
    )

    true_to_false_ids = [14280, 14305, 14321, 14335, 14338,  14352, 14355, 14403, 14407, 14412, 14413, 14418]
    df["MC_Answer"] = np.where(
        df["row_id"].isin(true_to_false_ids),
        df["MC_Answer"].str.replace(r"\( 9 \)", r"\( 6 \)"),
        df["MC_Answer"]
    )
    return df


def replace_duplicate_misc(df: pd.DataFrame) -> pd.DataFrame:
    """誤りのある誤答ラベルを修正する"""
    df["Misconception"] = df["Misconception"].replace({"Wrong_Fraction": "Wrong_fraction"})
    return df


def make_completion(df: pd.DataFrame) -> pd.DataFrame:
    """completion列を作成する"""
    df["Misconception"] = df["Misconception"].fillna("NA")
    df["completion"] = df["Category"] + ":" + df["Misconception"]
    return df


def add_is_correct(df: pd.DataFrame) -> pd.DataFrame:
    """正答かどうかのフラグを追加する"""
    idx = df.apply(lambda row: row["Category"].split("_")[0], axis=1) == "True"
    correct = df.loc[idx].copy()
    correct["count"] = correct.groupby(["QuestionId", "MC_Answer"]).MC_Answer.transform("count")
    correct = correct.sort_values("count", ascending=False)
    correct = correct.drop_duplicates(["QuestionId"])
    correct = correct[["QuestionId", "MC_Answer"]]
    correct["is_correct"] = 1

    df = df.merge(correct, on=["QuestionId", "MC_Answer"], how="left")
    df["is_correct"] = df["is_correct"].fillna(0)
    return df


def format_input(row) -> str:
    """入力テキストのフォーマット"""
    return CFG.prompt_format.format(
        QuestionText=row["QuestionText"],
        MC_Answer=row["MC_Answer"],
        Correct="Yes" if row["is_correct"] else "No",
        StudentExplanation=row["StudentExplanation"],
    )

In [ ]:
def load_soft_labels(soft_label_path: str, all_completions: list) -> dict:
    """ソフトラベルを読み込み、64カテゴリの確率分布に拡張する"""
    with open(soft_label_path, "rb") as f:
        soft_labels_raw = pickle.load(f)

    print(f"Loaded soft labels for {len(soft_labels_raw)} samples")

    # 64カテゴリの確率分布に拡張
    expanded_soft_labels = {}
    for idx, soft_label_dict in soft_labels_raw.items():
        # 64カテゴリの確率ベクトルを初期化（全て0.0）
        full_probs = np.zeros(len(all_completions), dtype=np.float32)

        # ソフトラベルに存在するカテゴリの確率を設定
        for category, prob in soft_label_dict.items():
            if category in all_completions:
                cat_idx = all_completions.index(category)
                full_probs[cat_idx] = prob

        # 確率の合計が1になるように正規化（ソフトラベルの確率の合計が1未満の場合）
        prob_sum = np.sum(full_probs)
        if prob_sum > 0:
            full_probs = full_probs / prob_sum

        expanded_soft_labels[idx] = full_probs

    return expanded_soft_labels

In [ ]:
# 学習データの読み込み
train = pd.read_csv(f"{CFG.comp_dataset_path}/train.csv")

# Fold分割
train = add_folds_by_qid_cat_misc(train, n_splits=5, random_state=42, fallback="pair")

# 既知の誤り修正
train = wrong_corrections(train)

# 重複するMisconceptionの統一
train = replace_duplicate_misc(train)

# completion列の作成
train = make_completion(train)

# 正解フラグの作成
train = add_is_correct(train)

# 入力プロンプトの作成
train["prompt"] = train.apply(format_input, axis=1)

# 全てのラベルを取得（64カテゴリ）
all_completions = sorted(train["completion"].unique().tolist())
print(f"Total categories: {len(all_completions)}")

In [ ]:
# ソフトラベルの読み込み
soft_labels = load_soft_labels(CFG.soft_label_path, all_completions)

# trainデータにソフトラベルを追加
train["soft_labels"] = train.index.map(lambda idx: soft_labels.get(idx, np.zeros(len(all_completions), dtype=np.float32)))

print(f"Added soft labels for {len(train)} samples")
print(f"Example soft label shape: {train['soft_labels'].iloc[0].shape}")
print(f"Example soft label sum: {np.sum(train['soft_labels'].iloc[0]):.4f}")

In [ ]:
# プロンプトの表示
print(train["prompt"].values[0])
print(train["completion"].values[0])

In [ ]:
# データセットの分割
train_df = train[train["fold"] != CFG.fold].sample(frac=1, random_state=CFG.seed).reset_index(drop=True)
val_df = train[train["fold"] == CFG.fold].reset_index(drop=True)

train_ds = Dataset.from_pandas(train_df[CFG.cols], preserve_index=False)
val_ds = Dataset.from_pandas(val_df[CFG.cols], preserve_index=False)

## 学習設定

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
# モデルの読み込み
model = AutoModelForCausalLM.from_pretrained(
    CFG.model_name,
    quantization_config=bnb_config,
    trust_remote_code=True,
    dtype=torch.bfloat16,
    # attn_implementation="flash_attention_2",
    device_map="auto",
)

# トークナイザーの読み込み
tokenizer = AutoTokenizer.from_pretrained(
    CFG.model_name,
    trust_remote_code=True,
    add_eos_token=True,
    add_bos_token=True,
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
eos_token_id = tokenizer.eos_token_id

In [ ]:
def add_completion_token(
        model: AutoModelForCausalLM,
        tokenizer: PreTrainedTokenizer,
        completions: list[str]
    ) -> PreTrainedTokenizer:
    special_tokens_dict = {"additional_special_tokens": completions}
    tokenizer.add_special_tokens(special_tokens_dict)
    print(f"Added {len(completions)} special tokens.")

    model.resize_token_embeddings(len(tokenizer))
    print(f"Resized model embeddings to {len(tokenizer)} tokens.")

    return model, tokenizer

In [ ]:
# 全てのラベルを保存
with open(f"{CFG.output_dir_path}/all_completions.json", "w", encoding="utf-8") as f:
    json.dump(all_completions, f, ensure_ascii=False, indent=2)

# 全てのラベルを特殊トークンとして追加
model, tokenizer = add_completion_token(model, tokenizer, all_completions)

In [ ]:
# wandbのログイン
wandb.login(key=userdata.get("WANDB_API_KEY"))
wandb.init(project=CFG.comp_name, name=CFG.exp_name)

In [ ]:
%%bash
cat <<'EOT' > ds_config_zero1.json
{
    "fp16": {
        "enabled": "auto",
        "loss_scale": 0,
        "loss_scale_window": 1000,
        "initial_scale_power": 16,
        "hysteresis": 2,
        "min_loss_scale": 1
    },
    "bf16": {
        "enabled": "auto"
    },
    "zero_optimization": {
        "stage": 1,
        "offload_optimizer": {
            "device": "none",
            "pin_memory": true
        }
    },
    "gradient_accumulation_steps": "auto",
    "gradient_clipping": "auto",
    "steps_per_print": 2000,
    "train_batch_size": "auto",
    "train_micro_batch_size_per_gpu": "auto",
    "wall_clock_breakdown": false
}
EOT

In [ ]:
# 学習の設定
sft_config = SFTConfig(
    output_dir=CFG.output_dir_path,
    do_train=True,
    do_eval=True,
    num_train_epochs=CFG.num_train_epochs,
    per_device_train_batch_size=CFG.per_device_train_batch_size,
    gradient_accumulation_steps=CFG.gradient_accumulation_steps,
    per_device_eval_batch_size=CFG.per_device_eval_batch_size,
    learning_rate=CFG.learning_rate,
    optim=CFG.optim_type,
    lr_scheduler_type=CFG.lr_scheduler_type,
    warmup_ratio=CFG.warmup_ratio,
    weight_decay=CFG.weight_decay,
    logging_dir=CFG.log_dir_path,
    eval_strategy="epoch",
    save_strategy="no",
    logging_steps=10,
    eval_steps=1,
    # save_steps=0.2,
    # save_total_limit=1,
    # metric_for_best_model="map@3",
    # greater_is_better=True,
    # load_best_model_at_end=True,
    max_length=CFG.max_len,
    max_grad_norm=1.0,
    report_to="wandb",
    bf16=True,
    fp16=False,
    bf16_full_eval=True,
    gradient_checkpointing=True,
    completion_only_loss=True,
    deepspeed="ds_config_zero1.json",
    dataset_num_proc=8,
    remove_unused_columns=False
)

In [ ]:
# 追加済みの全ラベルをIDに
new_token_ids = tokenizer.convert_tokens_to_ids(all_completions)

# LoRAの設定
lora_config = LoraConfig(
    r=CFG.lora_r,
    lora_alpha=CFG.lora_alpha,
    lora_dropout=CFG.lora_dropout,
    bias=CFG.lora_bias,
    target_modules=CFG.target_modules,
    task_type=CFG.task_type,
    modules_to_save=["lm_head"],
    trainable_token_indices={"embed_tokens": new_token_ids}
)

### カスタムトレーナー

In [ ]:
class SoftLabelCollator:
    "soft_labelsをcustom_trainerに渡す"
    def __init__(self, base_collator):
        self.base_collator = base_collator

    def __call__(self, examples):
        has_soft_labels = "soft_labels" in examples[0]
        if has_soft_labels:
            soft_labels = [ex["soft_labels"] for ex in examples]
        else:
            soft_labels = None

        # 既存のcollatorを通す
        batch = self.base_collator(examples)

        # soft_labels をそのまま追加
        if has_soft_labels:
            batch["soft_labels"] = soft_labels

        return batch

In [ ]:
class SoftLabelSFTTrainer(SFTTrainer):
    """ソフトラベル学習に対応したカスタムSFTTrainer"""

    def __init__(self, temperature=1.0, alpha=0.7, category_token_ids=None, **kwargs):
        super().__init__(**kwargs)
        self.temperature = temperature
        self.alpha = alpha
        self.category_token_ids = (torch.tensor(category_token_ids) if category_token_ids else None)
        self.kl_criterion = nn.KLDivLoss(reduction="batchmean")

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        """ソフトラベルとハードラベルの混合損失を計算"""

        # ソフトラベル取得
        soft_labels = inputs.pop("soft_labels", None)

        # 通常のforward pass
        outputs = model(**inputs)
        logits = outputs.logits

        # ------------------------------------------------------------
        # ソフトラベルとカテゴリトークンが指定されている場合のみ特殊処理
        # ------------------------------------------------------------
        if soft_labels is not None and self.category_token_ids is not None:
            labels = inputs.get("labels", None)
            if labels is None:
                return outputs.loss if not return_outputs else (outputs.loss, outputs)

            batch_size = labels.size(0)
            category_positions = []
            actual_category_labels = []

            # ------------------------------------------------------------
            # カテゴリトークンの位置を特定し、予測すべき位置を抽出
            # ------------------------------------------------------------
            for b in range(batch_size):
                sample_labels = labels[b]
                cat_mask = torch.zeros_like(sample_labels, dtype=torch.bool)

                for cat_id in self.category_token_ids:
                    cat_mask |= (sample_labels == cat_id.to(sample_labels.device))

                cat_positions = torch.nonzero(cat_mask, as_tuple=True)[0]

                if len(cat_positions) > 0:
                    # 最初のカテゴリトークンの1つ前を予測位置とする
                    pos = max(cat_positions[0].item() - 1, 0)
                    actual_category_labels.append(sample_labels[cat_positions[0]].item())
                else:
                    # カテゴリトークンが見つからない場合、最後の有効トークンを使う
                    print("カテゴリトークンが見つかりません")
                    valid_mask = sample_labels != -100
                    valid_positions = torch.nonzero(valid_mask, as_tuple=True)[0]
                    if len(valid_positions) > 0:
                        pos = valid_positions[-2].item() if len(valid_positions) > 1 else 0
                        actual_category_labels.append(sample_labels[valid_positions[-1]].item())
                    else:
                        pos = logits.size(1) - 1
                        actual_category_labels.append(-100)

                category_positions.append(pos)

            # ------------------------------------------------------------
            # 各サンプルの対象位置のlogitsを取得
            # ------------------------------------------------------------
            selected_logits = torch.stack([
                logits[i, pos, :] for i, pos in enumerate(category_positions)
            ])

            # ------------------------------------------------------------
            # デバッグ出力
            # ------------------------------------------------------------
            # if self.state.global_step % 10 == 0:
            #     with torch.no_grad():
            #         print(f"\n{'='*80}")
            #         print(f"[Step {self.state.global_step}] Token predictions at completion position:")

            #         for i in range(min(3, batch_size)):
            #             print(f"\n  Sample {i} (position={category_positions[i]}):")

            #             # --- (1) 全語彙からのトップ5 ---
            #             all_probs = F.softmax(selected_logits[i], dim=-1)
            #             top5_probs, top5_indices = torch.topk(all_probs, k=5, dim=-1)

            #             print("    Top-5 from ALL vocabulary:")
            #             for idx, prob in zip(top5_indices, top5_probs):
            #                 token_id = idx.item()
            #                 token_str = (
            #                     self.processing_class.decode([token_id]).strip()
            #                     if hasattr(self.processing_class, "decode")
            #                     else f"token_{token_id}"
            #                 )
            #                 marker = " [CAT]" if token_id in self.category_token_ids else ""
            #                 print(f"      {token_str}: {prob:.4f}{marker}")

            #             # --- (2) カテゴリトークンのみのトップ5 ---
            #             cat_logits = selected_logits[i][self.category_token_ids.to(selected_logits.device)]
            #             cat_probs = F.softmax(cat_logits, dim=-1)
            #             top5_cat_probs, top5_cat_indices = torch.topk(
            #                 cat_probs, k=min(5, cat_probs.size(-1)), dim=-1
            #             )

            #             print("    Top-5 CATEGORY tokens (Model Prediction):")
            #             for idx, prob in zip(top5_cat_indices, top5_cat_probs):
            #                 token_id = self.category_token_ids[idx].item()
            #                 token_str = (
            #                     self.processing_class.decode([token_id]).strip()
            #                     if hasattr(self.processing_class, "decode")
            #                     else f"cat_{token_id}"
            #                 )
            #                 print(f"      {token_str}: {prob:.4f}")

            #             # --- (3) ソフトラベルのトップ5 ---
            #             soft_label_tensor = torch.as_tensor(
            #                 soft_labels[i], dtype=torch.float32
            #             )
            #             top5_soft_probs, top5_soft_indices = torch.topk(
            #                 soft_label_tensor, k=min(5, len(soft_label_tensor))
            #             )

            #             print("    Top-5 SOFT LABELS (Target Distribution):")
            #             for idx, prob in zip(top5_soft_indices, top5_soft_probs):
            #                 token_id = self.category_token_ids[idx].item()
            #                 token_str = (
            #                     self.processing_class.decode([token_id]).strip()
            #                     if hasattr(self.processing_class, "decode")
            #                     else f"soft_{token_id}"
            #                 )
            #                 print(f"      {token_str}: {prob:.4f}")

            #             # --- (4) 正解ラベルを表示 ---
            #             if i < len(actual_category_labels) and actual_category_labels[i] != -100:
            #                 true_label_id = actual_category_labels[i]
            #                 true_label_str = (
            #                     self.processing_class.decode([true_label_id]).strip()
            #                     if hasattr(self.processing_class, "decode")
            #                     else f"label_{true_label_id}"
            #                 )

            #                 if true_label_id in self.category_token_ids:
            #                     cat_index = (self.category_token_ids == true_label_id).nonzero(as_tuple=True)[0]
            #                     if len(cat_index) > 0:
            #                         true_prob = cat_probs[cat_index[0]].item()
            #                         print(f"    TRUE LABEL: {true_label_str} (prob={true_prob:.4f})")
            #                     else:
            #                         print(f"    TRUE LABEL: {true_label_str}")
            #                 else:
            #                     print(f"    TRUE LABEL: {true_label_str} [NOT A CATEGORY TOKEN]")
            #             else:
            #                 print("    TRUE LABEL: Not found")

            # ------------------------------------------------------------
            # ソフトラベル損失（KL Divergence）の計算
            # ------------------------------------------------------------
            # 全語彙サイズを取得
            vocab_size = selected_logits.size(-1)

            # 全語彙に対するソフトラベルテンソルを初期化（全て0）
            soft_labels_full = torch.zeros(batch_size, vocab_size, device=selected_logits.device)

            # カテゴリトークンの位置にソフトラベルの値を設定
            for i in range(batch_size):
                for j, cat_id in enumerate(self.category_token_ids):
                    soft_labels_full[i, cat_id.item()] = soft_labels[i][j]

            # 全語彙に対してlog_softmaxを計算
            log_probs = F.log_softmax(selected_logits / self.temperature, dim=-1)

            # nn.KLDivLossを使用（全語彙に対して）
            kl_loss = self.kl_criterion(log_probs, soft_labels_full)
            kl_loss *= self.temperature ** 2

            # ------------------------------------------------------------
            # 損失の混合
            # ------------------------------------------------------------
            hard_loss = outputs.loss if outputs.loss is not None else 0
            total_loss = (1 - self.alpha) * hard_loss + self.alpha * kl_loss

            # if self.state.global_step % 10 == 0:
            #     print(f"\n  Losses - Hard: {hard_loss.item():.4f}, KL: {kl_loss.item():.4f}, Total: {total_loss.item():.4f}")
            #     print('=' * 80)

        else:
            # ソフトラベルを使用しない通常損失
            total_loss = outputs.loss

        return (total_loss, outputs) if return_outputs else total_loss


### 評価関数

In [ ]:
def preprocess_logits_for_metrics(logits, labels):
    """logitsから上位5件のトークンIDを抽出"""
    if isinstance(logits, tuple):
        logits = logits[0]

    # top-kの値とインデックスを取得
    k = 5
    _, topk_indices = torch.topk(
        logits,
        k=min(k, logits.size(-1)),
        dim=-1,
        largest=True,
        sorted=True
    )

    return topk_indices, labels

In [ ]:
def compute_metrics(eval_preds):
    """メトリクス（accuracy と MAP@3）を計算する関数。"""
    topk_preds, labels = eval_preds

    if isinstance(topk_preds, tuple):
        topk_preds = topk_preds[0]

    # -100 と eos を無視
    ignore_ids = {-100, eos_token_id}
    valid_labels_mask = ~np.isin(labels, list(ignore_ids))
    valid_labels = labels[valid_labels_mask]

    # preds とラベルを位置合わせ（1トークンずれ対策）
    shifted_mask = np.roll(valid_labels_mask, shift=-1, axis=1)
    shifted_mask[:, -1] = False
    aligned_topk_preds = topk_preds[shifted_mask]  # shape: (N, topk)

    # デコード
    decoded_topk_preds = [
        [tokenizer.decode([int(pred)]) for pred in preds_topk]
        for preds_topk in aligned_topk_preds
    ]
    decoded_labels = [tokenizer.decode([int(label)]) for label in valid_labels]

    # accuracy
    accuracy = np.mean([
        preds_topk[0].strip() == label.strip()
        for preds_topk, label in zip(decoded_topk_preds, decoded_labels)
    ])

    # MAP@3
    map3 = np.mean([
        sum((label.strip() == pred.strip()) / (rank+1)
            for rank, pred in enumerate(preds_topk))
        for preds_topk, label in zip(decoded_topk_preds, decoded_labels)
    ])

    return {"accuracy": float(accuracy), "map@3": float(map3)}

## モデルの学習

In [ ]:
# collatorの定義
base = DataCollatorForLanguageModeling(
    pad_token_id=tokenizer.pad_token_id,
    completion_only_loss=True,
    padding_free=False
)
collator = SoftLabelCollator(base)

In [ ]:
# カスタムトレーナーの設定
trainer = SoftLabelSFTTrainer(
    model=model,
    processing_class=tokenizer,
    args=sft_config,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    peft_config=lora_config,
    temperature=CFG.temperature,
    alpha=CFG.alpha,
    category_token_ids=new_token_ids,
    compute_metrics=compute_metrics,
    data_collator=collator,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics,
)

In [ ]:
for n, p in trainer.model.named_parameters():
    if "embed_tokens" in n or "lm_head" in n:
        print(n, p.requires_grad)

In [ ]:
# 学習
trainer.train()

In [ ]:
# モデルの保存
trainer.save_model(CFG.output_dir_path + "/model")
tokenizer.save_pretrained(CFG.output_dir_path + "/model")

In [ ]:
import time
from google.colab import runtime

def disconnect_runtime_after_timeout(timeout=3600):
    print(f"ランタイムが{timeout // 60}分後に自動で切断されます。")
    time.sleep(timeout)
    print("ランタイムを切断します...")
    runtime.unassign()

disconnect_runtime_after_timeout(600)